In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from itertools import product
from scipy.stats import unitary_group

In [2]:
# Configuración
np.random.seed(42)
torch.manual_seed(42)
n_qubits = 3
dim = 2 ** n_qubits

In [3]:
# ======================
# 1. Estado aleatorio
# ======================
def random_pure_state(n_qubits):
    dim = 2**n_qubits
    psi = np.random.randn(dim) + 1j * np.random.randn(dim)
    psi /= np.linalg.norm(psi)
    return psi

def density_matrix(psi):
    return np.outer(psi, np.conj(psi))

psi = random_pure_state(n_qubits)
rho_true = density_matrix(psi)

In [4]:
# ======================
# 2. Pauli Operators
# ======================
pauli_matrices = {
    "I": np.eye(2),
    "X": np.array([[0, 1], [1, 0]]),
    "Y": np.array([[0, -1j], [1j, 0]]),
    "Z": np.array([[1, 0], [0, -1]])
}

pauli_labels = ["X", "Y", "Z"]

def random_pauli_basis(n):
    return np.random.choice(pauli_labels, size=n)

def measurement_operator(basis):
    M = pauli_matrices[basis[0]]
    for p in basis[1:]:
        M = np.kron(M, pauli_matrices[p])
    return M

In [5]:
# ======================
# 3. Dataset
# ======================
def generate_dataset(rho, n_samples=20000):
    dataset = []
    for _ in range(n_samples):
        basis = random_pauli_basis(n_qubits)
        M = measurement_operator(basis)
        val = np.real(np.trace(rho @ M))
        dataset.append((basis, val))
    return dataset

dataset = generate_dataset(rho_true)

In [6]:
# ======================
# 4. Red Neuronal
# ======================
class PauliEncoder(nn.Module):
    def __init__(self, n_qubits):
        super().__init__()
        self.embed = nn.Embedding(3, 4)  # X, Y, Z → 0,1,2 → one-hot 4D
        self.net = nn.Sequential(
            nn.Linear(n_qubits * 4, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.embed(x)  # (batch, qubits, 4)
        x = x.view(x.shape[0], -1)
        return self.net(x).squeeze(-1)

def basis_to_tensor(basis_list):
    mapping = {"X": 0, "Y": 1, "Z": 2}
    return torch.tensor([[mapping[p] for p in basis] for basis in basis_list], dtype=torch.long)

basis_list, vals = zip(*dataset)
X_train = basis_to_tensor(basis_list)
y_train = torch.tensor(vals, dtype=torch.float32)

In [7]:
# ======================
# 5. Entrenamiento
# ======================
model = PauliEncoder(n_qubits)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

n_epochs = 3000

for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    preds = model(X_train)
    loss = loss_fn(preds, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item()}")

Epoch 0: Loss = 0.23161213099956512
Epoch 1000: Loss = 2.107495476283373e-15
Epoch 2000: Loss = 5.457474572015636e-16


In [8]:
# ======================
# 6. Reconstrucción
# ======================
# Expandimos la matriz densidad como:
# ρ = (1/2^n) * sum_r c_r * P_r, donde P_r ∈ base de Pauli

def all_pauli_strings(n):
    return list(product(pauli_labels, repeat=n))

def reconstruct_density_matrix(model, n_qubits):
    pauli_strings = all_pauli_strings(n_qubits)
    mapping = {"X": 0, "Y": 1, "Z": 2}
    X_test = torch.tensor([[mapping[p] for p in basis] for basis in pauli_strings], dtype=torch.long)
    with torch.no_grad():
        coeffs = model(X_test).numpy()

    rho_est = np.zeros((2**n_qubits, 2**n_qubits), dtype=complex)
    for basis, coeff in zip(pauli_strings, coeffs):
        M = measurement_operator(basis)
        rho_est += coeff * M

    rho_est /= 2**n_qubits
    return rho_est

rho_est = reconstruct_density_matrix(model, n_qubits)

In [9]:
# ======================
# 7. Fidelidad
# ======================
def fidelity(rho, sigma):
    sqrt_rho = scipy.linalg.sqrtm(rho)
    inner = sqrt_rho @ sigma @ sqrt_rho
    fid = np.real(np.trace(scipy.linalg.sqrtm(inner)))**2
    return fid

import scipy.linalg
fid = fidelity(rho_true, rho_est)
print(f"\nFidelidad estimada: {fid:.6f}")


Fidelidad estimada: 0.423483
